## 1. Start to get a feel for the data

When dealing with new datasets, it is always useful to look over any relevant documentation to get a feel for the data you will be handling.

Investigate the *Connected Nations 2022: Interactive report* (Ofcom, 2023a) which is available at:

   https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research/connected-nations-2022/interactive-report

One of the investigations you should do is to compare the fixed broadband coverage in your local authority with that of the UK as a whole. Take a single screenshot of what you have found and explain in no more than 100 words what the result says about broadband coverage in your local authority.

If you are not based in the UK then if possible choose a local authority with which you have some connection.

**You must include your screenshot in the `images/` directory.**

*(3 marks)*

Ofcom (2023a) *Connected Nations 2022: Interactive report*. Available at: https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research/connected-nations-2022/interactive-report (Accessed: 26 September 2023).

**Write your answer and include your image in this markdown cell**

<img src="images/screen2.png">

This indicates the coverage of premises by fixed broadband networks in Ealing compared to the UK. The bar graphs' shapes in Ealing resemble those of the UK, with '>= 10 Mbit/s' being the highest and <10 Mbit/s DL being the lowest. However, Full Fibre coverage in Ealing accounts for 31%, whereas the UK has 41%. Therefore, the Full Fibre  networks in Ealing have improved, though not to the same extent as across the entire UK. 

## 2. Store the data in a MongoDB database

The datafiles for fixed broadband coverage in the UK are found in the folder:
```
    2023J_TMA02_data/Ofcom_fixed
```
There are five csv files - one for each year - the year is indicated in the filename. Each csv file has an associated pdf which includes metadata. The datafiles and metadata pdfs were downloaded from Ofcom (2023b). The data is made available by Ofcom under the [Open Government Licence v3.0](https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/).

Your first task is to import the data, clean it, and store it in MongoDB. You will need to use '2019–2023 structural changes to local government in England' (2023) because you will need to be able to compare broadband coverage between different years.

When doing this you should consider that:

- Not all the csv have the same column names.
- Not all the csv have the same number of columns.
- There is ambiguous data.
- The year is not included in the file, only in the filename.

As well as the data imported from the csv files, each MongoDB document should have a field with a value referencing the year that the data was collected.

You might find it useful to use python's [glob](https://docs.python.org/3.8/library/glob.html) or [os](https://docs.python.org/3.8/library/os.html) modules which allow you to create a list of filenames in a directory.

Use comments and Markdown cells to discuss and justify any decisions you make when importing the data.

*(10 marks)*

'2019–2023 structural changes to local government in England' (2023) Wikipedia. Available at: https://en.wikipedia.org/wiki/2019%E2%80%932023_structural_changes_to_local_government_in_England (Accessed: 26 September 2023). 

Ofcom (2023b) *Connected Nations and infrastructure reports*. Available at: https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research (Accessed: 24 September 2023).

In [1]:
# Write your answer in this code cell.
# Use additional cells if necessary, including markdown
# cells to explain your decisions and code.

import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import chardet

import pymongo
import folium 


In [2]:
!ls 2023J_TMA02_data/Ofcom_fixed

201909_fixed_laua_coverage_r01.csv
202009_fixed_laua_coverage_r01.csv
202109_fixed_laua_coverage_r01.csv
202209-about-fixed-local-and-unitary-authority.pdf
202209_fixed_laua_coverage_r02.csv
202305_fixed_laua_coverage_r02.csv
cn-2020-about-fixed-coverage-local-and-unitary-authority.pdf
cn-2021-about-fixed-laua-coverage.pdf
connected-nations-2019-about-fixed-local-unitary-authority-area.pdf
fixed-coverage-local-unitary-authority-202305-v2.pdf


### 2019

In [3]:
# Check encoding
chardet.detect(open('2023J_TMA02_data/Ofcom_fixed/201909_fixed_laua_coverage_r01.csv','rb').read())

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

data file:
- encoding:  ascii 

In [4]:
# Read the CSV file into a DataFrame
df2019 = pd.read_csv('2023J_TMA02_data/Ofcom_fixed/201909_fixed_laua_coverage_r01.csv', encoding='ascii')


In [5]:
df2019.head()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
0,S12000033,ABERDEEN CITY,125441,125311,73.3,20.1,13.1,0.0,0.2,0.7,...,49,170,665,7275,73.3,20.1,0.0,0.1,0.5,5.8
1,S12000034,ABERDEENSHIRE,125085,124305,78.5,2.8,2.7,2.5,5.9,9.9,...,3163,4176,4993,10321,78.5,2.8,2.5,3.3,4.0,8.3
2,E07000223,ADUR,29770,29760,16.3,82.4,0.6,0.0,0.1,0.1,...,0,16,28,333,16.3,82.4,0.0,0.1,0.1,1.1
3,E07000026,ALLERDALE,51385,51284,89.8,1.7,1.7,1.2,2.6,3.6,...,619,704,550,2408,89.8,1.7,1.2,1.4,1.1,4.7
4,E07000032,AMBER VALLEY,60674,60596,67.4,25.3,22.1,0.1,0.9,2.1,...,89,460,705,3110,67.4,25.3,0.1,0.8,1.2,5.1


In [6]:
df2019.tail()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
377,E07000238,WYCHAVON,62475,62114,82.9,8.6,5.1,0.2,0.9,2.0,...,129,424,670,3761,82.9,8.6,0.2,0.7,1.1,6.0
378,E07000007,WYCOMBE,76433,76345,68.5,26.5,2.2,0.1,0.6,1.0,...,90,399,242,3050,68.5,26.5,0.1,0.5,0.3,4.0
379,E07000128,WYRE,56343,56280,88.4,5.9,5.5,0.1,0.3,0.7,...,69,114,199,2725,88.4,5.9,0.1,0.2,0.4,4.8
380,E07000239,WYRE FOREST,48100,48061,49.3,46.7,0.9,0.3,0.6,1.4,...,140,151,371,1196,49.3,46.7,0.3,0.3,0.8,2.5
381,E06000014,YORK,98735,98548,23.0,70.8,43.6,0.0,0.3,0.6,...,36,225,368,5293,23.0,70.8,0.0,0.2,0.4,5.4


In [7]:
# Data volume
df2019.shape

(382, 38)

In [8]:
# Data type
df2019.dtypes

laua                                                             object
laua_name                                                        object
All Premises                                                      int64
All Matched Premises                                              int64
SFBB availability (% premises)                                  float64
UFBB availability (% premises)                                  float64
Full Fibre availability (% premises)                            float64
% of premises unable to receive 2Mbit/s                         float64
% of premises unable to receive 5Mbit/s                         float64
% of premises unable to receive 10Mbit/s                        float64
% of premises unable to receive 30Mbit/s                        float64
% of premises below the USO                                     float64
% of premises with NGA                                          float64
% of premises able to receive decent broadband from FWA         

In [9]:
# Count null values
df2019.isnull().sum()

laua                                                            0
laua_name                                                       0
All Premises                                                    0
All Matched Premises                                            0
SFBB availability (% premises)                                  0
UFBB availability (% premises)                                  0
Full Fibre availability (% premises)                            0
% of premises unable to receive 2Mbit/s                         0
% of premises unable to receive 5Mbit/s                         0
% of premises unable to receive 10Mbit/s                        0
% of premises unable to receive 30Mbit/s                        0
% of premises below the USO                                     0
% of premises with NGA                                          0
% of premises able to receive decent broadband from FWA         0
% of premises able to receive SFBB from FWA                     0
Number of 

In [10]:
# Select only numerical columns
numerical_columns = df2019.select_dtypes(include=['number'])

# Use describe on numerical columns
numerical_description = numerical_columns.describe()

# Display the numerical description
display(numerical_description)


,All Premises,All Matched Premises,SFBB availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,% of premises unable to receive 30Mbit/s,% of premises below the USO,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
count,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,...,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000
mean,80678.520942,80532.005236,46.587435,46.857068,8.586649,0.408639,1.091361,2.078534,6.335340,0.643717,...,246.921466,430.123037,656.013089,3174.416230,46.587435,46.857068,0.408639,0.680366,0.984817,4.257330
std,52521.559617,52472.885870,25.255563,28.551654,10.401911,0.794339,1.663883,2.823999,5.478259,1.072613,...,449.014551,551.411163,718.525018,2971.853204,25.255563,28.551654,0.794339,0.922826,1.247035,3.413865
min,1681.000000,1678.000000,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,53.000000,1.100000,0.000000,0.000000,0.000000,0.000000,0.100000
25%,48100.250000,48070.250000,24.225000,18.450000,2.400000,0.000000,0.100000,0.400000,2.700000,0.100000,...,15.000000,72.250000,194.250000,1454.750000,24.225000,18.450000,0.000000,0.100000,0.300000,2.100000
50%,64867.500000,64782.500000,42.450000,51.800000,4.950000,0.100000,0.400000,1.000000,4.800000,0.200000,...,77.500000,225.000000,448.000000,2401.500000,42.450000,51.800000,0.100000,0.300000,0.600000,3.550000
75%,98485.750000,98413.250000,69.850000,72.175000,11.100000,0.400000,1.300000,2.500000,7.800000,0.700000,...,279.750000,536.000000,843.500000,3892.500000,69.850000,72.175000,0.400000,0.800000,1.200000,5.700000
max,469208.000000,468772.000000,97.800000,97.000000,97.000000,7.900000,13.500000,21.400000,46.000000,7.100000,...,3947.000000,4176.000000,4993.000000,25974.000000,97.800000,97.000000,7.900000,5.700000,11.400000,46.000000


### 2020

In [11]:
# Check encoding
chardet.detect(open('2023J_TMA02_data/Ofcom_fixed/202009_fixed_laua_coverage_r01.csv','rb').read())

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

data file:
- encoding:  ascii 

In [12]:
# Read the CSV file into a DataFrame
df2020 = pd.read_csv('2023J_TMA02_data/Ofcom_fixed/202009_fixed_laua_coverage_r01.csv', encoding='ascii')
df2020.head()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
0,S12000033,ABERDEEN CITY,126176,125948,94.6,49.0,41.6,34.9,34.9,0.0,...,55,153,673,5709,53.0,41.6,0.0,0.1,0.5,4.5
1,S12000034,ABERDEENSHIRE,126065,125176,82.9,7.2,7.0,6.9,6.9,2.6,...,3234,3865,4417,9188,75.8,7.0,2.6,3.1,3.5,7.3
2,E07000223,ADUR,29779,29755,98.8,85.8,85.6,0.6,0.6,0.0,...,0,10,24,294,13.2,85.6,0.0,0.0,0.1,1.0
3,E07000026,ALLERDALE,51647,51483,92.3,2.8,2.8,2.8,2.8,1.2,...,627,546,532,2085,89.5,2.8,1.2,1.1,1.0,4.0
4,E07000032,AMBER VALLEY,61134,60972,94.7,30.2,26.7,23.6,23.6,0.1,...,63,217,293,2524,68.0,26.7,0.1,0.4,0.5,4.1


In [13]:
df2020.tail()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
374,W06000006,WREXHAM,65867,65212,94.4,37.3,36.6,36.2,36.2,0.3,...,194,445,578,1825,57.7,36.6,0.3,0.7,0.9,2.8
375,E07000238,WYCHAVON,62536,62215,93.8,19.2,15.2,8.0,8.1,0.1,...,83,340,399,2750,78.5,15.2,0.1,0.5,0.6,4.4
376,E07000128,WYRE,56527,56411,95.1,22.7,22.7,22.3,22.3,0.1,...,61,113,194,2304,72.3,22.7,0.1,0.2,0.3,4.1
377,E07000239,WYRE FOREST,48237,48173,96.8,47.9,47.8,2.0,47.8,0.2,...,84,134,171,1104,49.0,47.8,0.2,0.3,0.4,2.3
378,E06000014,YORK,95949,95674,94.1,75.5,71.9,54.8,54.8,0.0,...,39,170,565,4587,22.3,71.9,0.0,0.2,0.6,4.8


In [14]:
# Data volume
df2020.shape

(379, 40)

In [15]:
# Data types
df2020.dtypes

laua                                                             object
laua_name                                                        object
All Premises                                                      int64
All Matched Premises                                              int64
SFBB availability (% premises)                                  float64
UFBB (100Mbit/s) availability (% premises)                      float64
UFBB availability (% premises)                                  float64
Full Fibre availability (% premises)                            float64
Gigabit availability (% premises)                               float64
% of premises unable to receive 2Mbit/s                         float64
% of premises unable to receive 5Mbit/s                         float64
% of premises unable to receive 10Mbit/s                        float64
% of premises unable to receive 30Mbit/s                        float64
% of premises below the USO                                     

In [16]:
# Count null values
df2020.isnull().sum()

laua                                                            0
laua_name                                                       0
All Premises                                                    0
All Matched Premises                                            0
SFBB availability (% premises)                                  0
UFBB (100Mbit/s) availability (% premises)                      0
UFBB availability (% premises)                                  0
Full Fibre availability (% premises)                            0
Gigabit availability (% premises)                               0
% of premises unable to receive 2Mbit/s                         0
% of premises unable to receive 5Mbit/s                         0
% of premises unable to receive 10Mbit/s                        0
% of premises unable to receive 30Mbit/s                        0
% of premises below the USO                                     0
% of premises with NGA                                          0
% of premi

In [17]:
# Select only numerical columns
numerical_columns = df2020.select_dtypes(include=['number'])

# Use describe on numerical columns
numerical_description = numerical_columns.describe()

# Display the numerical description
display(numerical_description)

,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
count,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,...,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000,379.000000
mean,82073.034301,81814.664908,94.187071,54.782850,52.300000,14.926121,21.593931,0.388654,0.984433,1.832190,...,233.870712,377.614776,569.860158,2740.870712,41.887335,52.300000,0.388654,0.592612,0.845383,3.613720
std,54691.926020,54579.787696,5.414738,28.169708,28.214368,14.855957,22.950617,0.799870,1.621302,2.711307,...,454.429163,517.932346,664.281431,2705.698190,25.349735,28.214368,0.799870,0.873295,1.182164,3.106772
min,1677.000000,1666.000000,56.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,21.000000,0.900000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48481.000000,48342.000000,93.000000,30.500000,26.100000,4.550000,5.350000,0.000000,0.100000,0.400000,...,14.000000,64.000000,158.500000,1242.500000,19.050000,26.100000,0.000000,0.100000,0.200000,1.700000
50%,65648.000000,65115.000000,95.900000,61.700000,58.100000,10.200000,12.700000,0.100000,0.300000,0.800000,...,67.000000,188.000000,355.000000,2024.000000,36.100000,58.100000,0.100000,0.200000,0.500000,2.800000
75%,99125.000000,98969.000000,97.600000,78.950000,77.750000,20.100000,25.500000,0.400000,1.000000,2.000000,...,232.500000,458.000000,728.500000,3246.500000,64.550000,77.750000,0.400000,0.700000,1.000000,4.800000
max,474257.000000,473084.000000,99.600000,97.500000,97.500000,97.500000,97.500000,8.400000,13.800000,20.700000,...,4196.000000,3865.000000,4417.000000,27281.000000,97.600000,97.500000,8.400000,5.500000,11.100000,43.300000


In [18]:
# Find columns in df2019 but not in df2020
columns_only_in_df2019 = set(df2019.columns) - set(df2020.columns)

# Find columns in df2020 but not in df2019
columns_only_in_df2020 = set(df2020.columns) - set(df2019.columns)

# Display the results
print("Columns only in df2019:", columns_only_in_df2019)
print("Columns only in df2020:", columns_only_in_df2020)

Columns only in df2019: {'% of premises able to receive SFBB from FWA', 'Number of premises able to receive SFBB from FWA'}
Columns only in df2020: {'Gigabit availability (% premises)', 'Number of premises with UFBB (100Mbit/s) availability', 'UFBB (100Mbit/s) availability (% premises)', 'Number of premises with Gigabit availability'}


### 2021

In [19]:
# Check encoding
chardet.detect(open('2023J_TMA02_data/Ofcom_fixed/202109_fixed_laua_coverage_r01.csv','rb').read())

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

data file:
- encoding:  ascii 

In [20]:
# Read the CSV file into a DataFrame
df2021 = pd.read_csv('2023J_TMA02_data/Ofcom_fixed/202109_fixed_laua_coverage_r01.csv', encoding='ascii')
df2021.head()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
0,S12000033,ABERDEEN CITY,127714,126771,94.7,66.6,62.1,58.4,58.4,0.1,...,67,144,417,5205,32.5,62.1,0.1,0.1,0.3,4.1
1,S12000034,ABERDEENSHIRE,126481,125378,82.8,13.8,13.7,13.7,13.7,2.5,...,3202,3984,4445,8986,69.1,13.7,2.5,3.1,3.5,7.1
2,E07000223,ADUR,29884,29793,98.6,85.9,85.6,1.8,1.8,0.0,...,4,9,23,281,13.0,85.6,0.0,0.0,0.1,0.9
3,E07000026,ALLERDALE,51933,51622,92.3,3.4,3.4,3.4,3.4,1.1,...,595,503,514,2088,88.9,3.4,1.1,1.0,1.0,4.0
4,E07000032,AMBER VALLEY,61555,61161,95.1,31.4,27.9,25.2,25.2,0.1,...,61,129,189,2266,67.2,27.9,0.1,0.2,0.3,3.7


In [21]:
df2021.tail()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
369,W06000006,WREXHAM,66192,65306,94.6,42.0,41.4,41.0,41.0,0.2,...,165,388,550,1614,53.1,41.4,0.2,0.6,0.8,2.4
370,E07000238,WYCHAVON,63359,62695,94.2,23.8,20.0,12.8,17.5,0.2,...,105,362,293,2245,74.2,20.0,0.2,0.6,0.5,3.5
371,E07000128,WYRE,57413,57099,95.4,46.4,46.4,46.3,46.3,0.1,...,46,94,221,1981,49.0,46.4,0.1,0.2,0.4,3.5
372,E07000239,WYRE FOREST,48472,48204,96.7,48.5,48.1,2.6,48.1,0.1,...,45,96,182,1022,48.6,48.1,0.1,0.2,0.4,2.1
373,E06000014,YORK,96147,95638,94.2,77.5,74.5,60.4,72.0,0.0,...,41,141,486,4361,19.7,74.5,0.0,0.1,0.5,4.5


In [22]:
# Data volume
df2021.shape

(374, 40)

In [23]:
# Count null values
df2021.isnull().sum()

laua                                                            0
laua_name                                                       0
All Premises                                                    0
All Matched Premises                                            0
SFBB availability (% premises)                                  0
UFBB (100Mbit/s) availability (% premises)                      0
UFBB availability (% premises)                                  0
Full Fibre availability (% premises)                            0
Gigabit availability (% premises)                               0
% of premises unable to receive 2Mbit/s                         0
% of premises unable to receive 5Mbit/s                         0
% of premises unable to receive 10Mbit/s                        0
% of premises unable to receive 30Mbit/s                        0
% of premises below the USO                                     0
% of premises with NGA                                          0
% of premi

In [24]:
# Select only numerical columns
numerical_columns2 = df2021.select_dtypes(include=['number'])

# Use describe on numerical columns
numerical_description2 = numerical_columns2.describe()

# Display the numerical description
display(numerical_description2)

,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,...,374.000000,374.000000,374.00000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,83731.264706,83184.727273,94.344652,59.760963,57.726203,23.960963,38.224599,0.365775,0.913102,1.681551,...,223.211230,358.179144,523.78877,2506.422460,36.614439,57.726203,0.365775,0.545989,0.763904,3.281283
std,55452.666179,55099.509244,5.246693,26.495761,26.509567,18.610830,28.511055,0.764637,1.543772,2.568474,...,440.084224,499.915776,627.13287,2550.335573,23.804715,26.509567,0.764637,0.824213,1.113291,2.975075
min,1683.000000,1661.000000,56.500000,1.100000,1.100000,0.900000,0.900000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,20.000000,0.900000,1.100000,0.000000,0.000000,0.000000,0.000000
25%,49355.500000,49097.750000,93.225000,39.275000,36.875000,10.225000,12.900000,0.000000,0.100000,0.300000,...,15.000000,65.750000,146.25000,1108.000000,16.225000,36.875000,0.000000,0.100000,0.200000,1.500000
50%,66536.000000,66171.500000,95.900000,67.750000,65.600000,19.500000,29.900000,0.100000,0.300000,0.800000,...,63.500000,185.500000,325.00000,1832.000000,29.950000,65.600000,0.100000,0.200000,0.400000,2.500000
75%,101537.000000,100732.500000,97.500000,82.275000,80.500000,33.575000,66.625000,0.300000,0.900000,1.775000,...,202.000000,423.000000,649.75000,3020.250000,54.350000,80.500000,0.300000,0.600000,0.800000,4.575000
max,474961.000000,471159.000000,99.500000,97.600000,97.600000,97.600000,97.600000,8.200000,13.500000,20.100000,...,4109.000000,3984.000000,4445.00000,29149.000000,94.800000,97.600000,8.200000,5.300000,11.000000,42.000000


In [25]:
# Find columns in df2019 but not in df2020
columns_only_in_df2019 = set(df2019.columns) - set(df2021.columns)

# Find columns in df2021 but not in df2019
columns_only_in_df2021 = set(df2021.columns) - set(df2019.columns)

# Display the results
print("Columns only in df2019:", columns_only_in_df2019)
print("Columns only in df2021:", columns_only_in_df2021)

Columns only in df2019: {'% of premises able to receive SFBB from FWA', 'Number of premises able to receive SFBB from FWA'}
Columns only in df2021: {'Gigabit availability (% premises)', 'Number of premises with UFBB (100Mbit/s) availability', 'UFBB (100Mbit/s) availability (% premises)', 'Number of premises with Gigabit availability'}


In [26]:
# Find columns in df2020 but not in df2021
columns_only_in_df2020 = set(df2020.columns) - set(df2021.columns)

# Find columns in df2021 but not in df2020
columns_only_in_df2021 = set(df2021.columns) - set(df2020.columns)

# Display the results
print("Columns only in df2019:", columns_only_in_df2020)
print("Columns only in df2021:", columns_only_in_df2021)

Columns only in df2019: set()
Columns only in df2021: set()


it has the same columns in 2021 and 2020

### 2022

In [27]:
# Check encoding
chardet.detect(open('2023J_TMA02_data/Ofcom_fixed/202209_fixed_laua_coverage_r02.csv','rb').read())

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [28]:
# Read the CSV file into a DataFrame
df2022 = pd.read_csv('2023J_TMA02_data/Ofcom_fixed/202209_fixed_laua_coverage_r02.csv', encoding='ascii')

In [29]:
df2022.head()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
0,S12000033,ABERDEEN CITY,128708,128294,95.8,79.1,77.1,74.4,74.4,0.0,...,59,119,370,4411,18.8,77.1,0.0,0.1,0.3,3.4
1,S12000034,ABERDEENSHIRE,127941,127265,84.2,20.6,20.5,20.5,20.5,1.8,...,2330,3450,4660,9083,63.7,20.5,1.8,2.7,3.6,7.1
2,E07000223,ADUR,29971,29920,99.1,91.0,91.0,54.5,90.1,0.0,...,2,6,39,186,8.1,91.0,0.0,0.0,0.1,0.6
3,E07000026,ALLERDALE,52309,52133,92.7,5.4,5.4,5.4,5.4,1.2,...,630,456,455,2093,87.3,5.4,1.2,0.9,0.9,4.0
4,E07000032,AMBER VALLEY,62170,61902,96.1,49.4,46.2,43.6,43.7,0.1,...,58,103,126,1855,49.9,46.2,0.1,0.2,0.2,3.0


In [30]:
df2022.tail()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
369,W06000006,WREXHAM,66672,65735,95.2,48.6,48.1,47.6,47.6,0.2,...,132,349,447,1334,47.1,48.1,0.2,0.5,0.7,2.0
370,E07000238,WYCHAVON,64057,63530,95.1,35.8,31.0,29.4,30.7,0.1,...,67,272,220,2029,64.1,31.0,0.1,0.4,0.3,3.2
371,E07000128,WYRE,58069,57900,97.0,60.3,60.3,60.2,60.3,0.1,...,42,99,122,1320,36.7,60.3,0.1,0.2,0.2,2.3
372,E07000239,WYRE FOREST,48894,48679,97.3,55.7,55.7,10.3,55.4,0.1,...,49,87,114,872,41.5,55.7,0.1,0.2,0.2,1.8
373,E06000014,YORK,96526,96317,94.7,75.8,72.6,52.3,70.0,0.0,...,42,147,505,4175,22.2,72.6,0.0,0.2,0.5,4.3


In [31]:
# Data volume
df2022.shape

(374, 40)

In [32]:
# Count null values
df2022.isnull().sum()

laua                                                            0
laua_name                                                       0
All Premises                                                    0
All Matched Premises                                            0
SFBB availability (% premises)                                  0
UFBB (100Mbit/s) availability (% premises)                      0
UFBB availability (% premises)                                  0
Full Fibre availability (% premises)                            0
Gigabit availability (% premises)                               0
% of premises unable to receive 2Mbit/s                         0
% of premises unable to receive 5Mbit/s                         0
% of premises unable to receive 10Mbit/s                        0
% of premises unable to receive 30Mbit/s                        0
% of premises below the USO                                     0
% of premises with NGA                                          0
% of premi

In [33]:
# Select only numerical columns
numerical_columns3 = df2022.select_dtypes(include=['number'])

# Use describe on numerical columns
numerical_description3 = numerical_columns3.describe()

# Display the numerical description
display(numerical_description3)

,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,...,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,84761.181818,84349.454545,95.325668,67.605615,66.110428,38.207754,64.590642,0.291979,0.736631,1.362299,...,184.927807,293.545455,433.122995,2170.133690,29.213904,66.110428,0.291979,0.437166,0.624332,2.804813
std,56035.842818,55767.364674,4.584913,22.297820,22.464340,20.553333,22.199580,0.560008,1.171155,2.090665,...,338.461553,402.030606,548.774815,2335.513567,20.007487,22.464340,0.560008,0.647729,1.005039,2.783777
min,1686.000000,1669.000000,58.700000,1.600000,1.600000,1.600000,1.600000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,10.000000,0.800000,1.600000,0.000000,0.000000,0.000000,0.000000
25%,50147.500000,49881.250000,94.500000,52.950000,50.775000,22.300000,49.900000,0.000000,0.100000,0.300000,...,14.250000,52.250000,115.250000,935.500000,12.400000,50.775000,0.000000,0.100000,0.200000,1.300000
50%,67870.000000,67463.500000,96.750000,74.400000,72.200000,37.800000,70.350000,0.100000,0.300000,0.650000,...,55.000000,153.000000,265.000000,1522.500000,23.800000,72.200000,0.100000,0.200000,0.350000,2.100000
75%,102291.250000,102037.000000,98.000000,85.975000,84.800000,51.275000,83.025000,0.300000,0.800000,1.400000,...,186.750000,363.000000,504.250000,2482.000000,43.200000,84.800000,0.300000,0.500000,0.700000,3.600000
max,477617.000000,475094.000000,99.500000,97.700000,97.700000,97.700000,97.700000,3.900000,7.100000,17.300000,...,2526.000000,3450.000000,4660.000000,26434.000000,95.100000,97.700000,3.900000,4.300000,10.900000,39.300000


In [34]:
# Find columns in df2019 but not in df2020
columns_only_in_df2019 = set(df2019.columns) - set(df2022.columns)

# Find columns in df2020 but not in df2019
columns_only_in_df2022 = set(df2022.columns) - set(df2019.columns)

# Display the results
print("Columns only in df2019:", columns_only_in_df2019)
print("Columns only in df2020:", columns_only_in_df2022)

Columns only in df2019: {'% of premises able to receive SFBB from FWA', 'Number of premises able to receive SFBB from FWA'}
Columns only in df2020: {'Gigabit availability (% premises)', 'Number of premises with UFBB (100Mbit/s) availability', 'UFBB (100Mbit/s) availability (% premises)', 'Number of premises with Gigabit availability'}


In [35]:
# Find columns in df2021 but not in df2020
columns_only_in_df2021 = set(df2021.columns) - set(df2022.columns)

# Find columns in df2020 but not in df2021
columns_only_in_df2022 = set(df2022.columns) - set(df2021.columns)

# Display the results
print("Columns only in df2021:", columns_only_in_df2021)
print("Columns only in df2020:", columns_only_in_df2022)

Columns only in df2021: set()
Columns only in df2020: set()


Columns: 2020, 2021, 2022 are the same 

### 2023

In [36]:
# Check encoding
chardet.detect(open('2023J_TMA02_data/Ofcom_fixed/202305_fixed_laua_coverage_r02.csv','rb').read())

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [37]:
# Read the CSV file into a DataFrame
df2023 = pd.read_csv('2023J_TMA02_data/Ofcom_fixed/202305_fixed_laua_coverage_r02.csv', encoding='ascii')
df2023.head()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
0,S12000033,ABERDEEN CITY,129315,129197,97.2,84.8,83.9,83.0,83.0,0.0,...,58,151,222,3130,13.2,83.9,0.0,0.1,0.2,2.4
1,S12000034,ABERDEENSHIRE,128408,128070,85.9,25.5,25.5,25.4,25.4,1.7,...,2214,3193,4296,8071,60.4,25.5,1.7,2.5,3.3,6.3
2,E07000223,ADUR,29985,29953,99.1,92.8,92.8,65.4,92.8,0.0,...,0,9,31,186,6.3,92.8,0.0,0.0,0.1,0.6
3,E07000026,ALLERDALE,52482,52364,93.1,6.0,6.0,6.0,6.0,1.2,...,617,440,422,2000,87.2,6.0,1.2,0.8,0.8,3.8
4,E07000032,AMBER VALLEY,62512,62430,97.2,62.4,60.6,59.0,59.1,0.1,...,42,115,88,1415,36.6,60.6,0.1,0.2,0.1,2.3


In [38]:
df2023.head()

,laua,laua_name,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
0,S12000033,ABERDEEN CITY,129315,129197,97.2,84.8,83.9,83.0,83.0,0.0,...,58,151,222,3130,13.2,83.9,0.0,0.1,0.2,2.4
1,S12000034,ABERDEENSHIRE,128408,128070,85.9,25.5,25.5,25.4,25.4,1.7,...,2214,3193,4296,8071,60.4,25.5,1.7,2.5,3.3,6.3
2,E07000223,ADUR,29985,29953,99.1,92.8,92.8,65.4,92.8,0.0,...,0,9,31,186,6.3,92.8,0.0,0.0,0.1,0.6
3,E07000026,ALLERDALE,52482,52364,93.1,6.0,6.0,6.0,6.0,1.2,...,617,440,422,2000,87.2,6.0,1.2,0.8,0.8,3.8
4,E07000032,AMBER VALLEY,62512,62430,97.2,62.4,60.6,59.0,59.1,0.1,...,42,115,88,1415,36.6,60.6,0.1,0.2,0.1,2.3


In [39]:
# Data volume
df2023.shape

(374, 40)

In [40]:
# Count null values
df2023.isnull().sum()

laua                                                            0
laua_name                                                       0
All Premises                                                    0
All Matched Premises                                            0
SFBB availability (% premises)                                  0
UFBB (100Mbit/s) availability (% premises)                      0
UFBB availability (% premises)                                  0
Full Fibre availability (% premises)                            0
Gigabit availability (% premises)                               0
% of premises unable to receive 2Mbit/s                         0
% of premises unable to receive 5Mbit/s                         0
% of premises unable to receive 10Mbit/s                        0
% of premises unable to receive 30Mbit/s                        0
% of premises below the USO                                     0
% of premises with NGA                                          0
% of premi

In [41]:
# Select only numerical columns
numerical_columns4 = df2023.select_dtypes(include=['number'])

# Use describe on numerical columns
numerical_description4 = numerical_columns4.describe()

# Display the numerical description
display(numerical_description4)

,All Premises,All Matched Premises,SFBB availability (% premises),UFBB (100Mbit/s) availability (% premises),UFBB availability (% premises),Full Fibre availability (% premises),Gigabit availability (% premises),% of premises unable to receive 2Mbit/s,% of premises unable to receive 5Mbit/s,% of premises unable to receive 10Mbit/s,...,Number of premises with 0<2Mbit/s download speed,Number of premises with 2<5Mbit/s download speed,Number of premises with 5<10Mbit/s download speed,Number of premises with 10<30Mbit/s download speed,% of premises with 30<300Mbit/s download speed,% of premises with >=300Mbit/s download speed,% of premises with 0<2Mbit/s download speed,% of premises with 2<5Mbit/s download speed,% of premises with 5<10Mbit/s download speed,% of premises with 10<30Mbit/s download speed
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,...,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,85116.336898,84886.005348,96.072995,71.809091,70.550802,47.627807,69.724866,0.274064,0.679412,1.239305,...,175.125668,272.219251,392.620321,1865.080214,25.524064,70.550802,0.274064,0.397594,0.560695,2.404011
std,56252.163193,56070.215034,4.211346,20.414938,20.496678,20.763319,20.390288,0.504081,1.073881,1.937932,...,304.743680,378.581337,508.478333,2071.790679,18.175820,20.496678,0.504081,0.604247,0.939757,2.639301
min,1689.000000,1678.000000,59.100000,1.800000,1.800000,1.800000,1.800000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,26.000000,0.200000,1.800000,0.000000,0.000000,0.000000,0.000000
25%,50502.250000,50250.000000,95.400000,59.075000,57.550000,32.175000,57.475000,0.000000,0.100000,0.300000,...,19.000000,54.250000,100.000000,803.000000,10.575000,57.550000,0.000000,0.100000,0.100000,1.100000
50%,68136.500000,67915.000000,97.300000,77.050000,75.350000,48.450000,74.400000,0.100000,0.300000,0.600000,...,57.500000,137.000000,240.000000,1267.500000,21.250000,75.350000,0.100000,0.200000,0.300000,1.700000
75%,102462.000000,102371.250000,98.300000,88.175000,87.200000,62.050000,86.850000,0.300000,0.700000,1.200000,...,185.750000,334.000000,467.750000,2143.750000,36.600000,87.200000,0.300000,0.400000,0.600000,3.075000
max,478734.000000,476604.000000,99.800000,98.400000,98.400000,98.400000,98.400000,3.300000,7.300000,16.700000,...,2568.000000,3193.000000,4296.000000,24111.000000,95.900000,98.400000,3.300000,4.300000,10.600000,39.800000


In [42]:
# Find columns in df2019 but not in df2023
columns_only_in_df2019 = set(df2019.columns) - set(df2023.columns)

# Find columns in df2023 but not in df2019
columns_only_in_df2023 = set(df2023.columns) - set(df2019.columns)

# Display the results
print("Columns only in df2019:", columns_only_in_df2019)
print("Columns only in df2023:", columns_only_in_df2023)

Columns only in df2019: {'% of premises able to receive SFBB from FWA', 'Number of premises able to receive SFBB from FWA'}
Columns only in df2023: {'Gigabit availability (% premises)', 'Number of premises with UFBB (100Mbit/s) availability', 'UFBB (100Mbit/s) availability (% premises)', 'Number of premises with Gigabit availability'}


In [43]:
# Find columns in df2020 but not in df2023
columns_only_in_df2020 = set(df2020.columns) - set(df2023.columns)

# Find columns in df2023 but not in df2020
columns_only_in_df2023 = set(df2023.columns) - set(df2020.columns)

# Display the results
print("Columns only in df2020:", columns_only_in_df2020)
print("Columns only in df2023:", columns_only_in_df2023)

Columns only in df2020: set()
Columns only in df2023: set()


In [44]:
df2019.dtypes

laua                                                             object
laua_name                                                        object
All Premises                                                      int64
All Matched Premises                                              int64
SFBB availability (% premises)                                  float64
UFBB availability (% premises)                                  float64
Full Fibre availability (% premises)                            float64
% of premises unable to receive 2Mbit/s                         float64
% of premises unable to receive 5Mbit/s                         float64
% of premises unable to receive 10Mbit/s                        float64
% of premises unable to receive 30Mbit/s                        float64
% of premises below the USO                                     float64
% of premises with NGA                                          float64
% of premises able to receive decent broadband from FWA         

The dataframe of df2019 has not the column related to Gigabit('Gigabit availability (% premises)' and 'Number of premises with Gigabit availability'). 

The columns names of df2019: 'Number of premises able to receive SFBB from FWA' '% of premises able to receive SFBB from FWA' of df2019 are the same as 'SFBB availability (% premises)' and 'Number of premises with SFBB availability' respectively. 
In addition, the columns from 'UFBB availability (% premises)' and 'Number of premises with UFBB (100Mbit/s) availability' from df2019 are the same as 'UFBB (100Mbit/s) availability (% premises)' and 'Number of premises with UFBB (100Mbit/s) availability' from other dataframes (2020 - 2023).

Therefore, the names of df2019 are changed  

In [45]:
df2019.rename(columns={'Number of premises able to receive SFBB from FWA':'Number of premises with SFBB availability',
                       '% of premises able to receive SFBB from FWA': 'SFBB availability (% premises)',
                       'UFBB availability (% premises)': 'UFBB (100Mbit/s) availability (% premises)', 
                       'Number of premises with UFBB (100Mbit/s) availability' : 'Number of premises with UFBB (100Mbit/s) availability'
                      },
              inplace=True)

In [46]:
# Data types
df2023.dtypes

laua                                                             object
laua_name                                                        object
All Premises                                                      int64
All Matched Premises                                              int64
SFBB availability (% premises)                                  float64
UFBB (100Mbit/s) availability (% premises)                      float64
UFBB availability (% premises)                                  float64
Full Fibre availability (% premises)                            float64
Gigabit availability (% premises)                               float64
% of premises unable to receive 2Mbit/s                         float64
% of premises unable to receive 5Mbit/s                         float64
% of premises unable to receive 10Mbit/s                        float64
% of premises unable to receive 30Mbit/s                        float64
% of premises below the USO                                     

In [47]:
# Find 'laua_name' values in 2019 but not in 2020
laua_names_only_in_2019 = set(df2019['laua_name']) - set(df2020['laua_name'])

# Display the results
display("laua_name values only in 2019:", laua_names_only_in_2019)

# Find 'laua_name' values in 2020 but not in 2019
laua_names_only_in_2020 = set(df2020['laua_name']) - set(df2019['laua_name'])

# Display the results
display("laua_name values only in 2020:", laua_names_only_in_2020)


'laua_name values only in 2019:'

{'AYLESBURY VALE', 'CHILTERN', 'SOUTH BUCKS', 'WYCOMBE'}

'laua_name values only in 2020:'

{'BUCKINGHAMSHIRE'}

In [48]:
# Find 'laua_name' values in 2019 but not in 2021
laua_names_only_in_2019 = set(df2019['laua_name']) - set(df2021['laua_name'])

# Display the results
display("laua_name values only in 2019:", laua_names_only_in_2019)

# Find 'laua_name' values in 2021 but not in 2019
laua_names_only_in_2021 = set(df2021['laua_name']) - set(df2019['laua_name'])

# Display the results
display("laua_name values only in 2021:", laua_names_only_in_2021)

'laua_name values only in 2019:'

{'AYLESBURY VALE',
 'CHILTERN',
 'CORBY',
 'DAVENTRY',
 'EAST NORTHAMPTONSHIRE',
 'KETTERING',
 'NORTHAMPTON',
 'SOUTH BUCKS',
 'SOUTH NORTHAMPTONSHIRE',
 'WELLINGBOROUGH',
 'WYCOMBE'}

'laua_name values only in 2021:'

{'BUCKINGHAMSHIRE', 'NORTH NORTHAMPTONSHIRE', 'WEST NORTHAMPTONSHIRE'}

In [49]:
# Find 'laua_name' values in 2019 but not in 2022
laua_names_only_in_2019 = set(df2019['laua_name']) - set(df2022['laua_name'])

# Display the results
display("laua_name values only in 2019:", laua_names_only_in_2019)

# Find 'laua_name' values in 2022 but not in 2019
laua_names_only_in_2022 = set(df2022['laua_name']) - set(df2019['laua_name'])

# Display the results
display("laua_name values only in 2023:", laua_names_only_in_2022)

'laua_name values only in 2019:'

{'AYLESBURY VALE',
 'CHILTERN',
 'CORBY',
 'DAVENTRY',
 'EAST NORTHAMPTONSHIRE',
 'KETTERING',
 'NORTHAMPTON',
 'SOUTH BUCKS',
 'SOUTH NORTHAMPTONSHIRE',
 'WELLINGBOROUGH',
 'WYCOMBE'}

'laua_name values only in 2023:'

{'BUCKINGHAMSHIRE', 'NORTH NORTHAMPTONSHIRE', 'WEST NORTHAMPTONSHIRE'}

In [50]:
# Find 'laua_name' values in 2019 but not in 2023
laua_names_only_in_2019 = set(df2019['laua_name']) - set(df2023['laua_name'])

# Display the results
display("laua_name values only in 2019:", laua_names_only_in_2019)

# Find 'laua_name' values in 2023 but not in 2019
laua_names_only_in_2023 = set(df2023['laua_name']) - set(df2019['laua_name'])

# Display the results
display("laua_name values only in 2023:", laua_names_only_in_2023)

'laua_name values only in 2019:'

{'AYLESBURY VALE',
 'CHILTERN',
 'CORBY',
 'DAVENTRY',
 'EAST NORTHAMPTONSHIRE',
 'KETTERING',
 'NORTHAMPTON',
 'SOUTH BUCKS',
 'SOUTH NORTHAMPTONSHIRE',
 'WELLINGBOROUGH',
 'WYCOMBE'}

'laua_name values only in 2023:'

{'BUCKINGHAMSHIRE', 'NORTH NORTHAMPTONSHIRE', 'WEST NORTHAMPTONSHIRE'}

In [51]:
# Find 'laua_name' values in 2020 but not in 2021
laua_names_only_in_2020 = set(df2020['laua_name']) - set(df2021['laua_name'])

# Display the results
display("laua_name values only in 2020:", laua_names_only_in_2020)

# Find 'laua_name' values in 2021 but not in 2020
laua_names_only_in_2021 = set(df2021['laua_name']) - set(df2020['laua_name'])

# Display the results
display("laua_name values only in 2023:", laua_names_only_in_2021)

'laua_name values only in 2020:'

{'CORBY',
 'DAVENTRY',
 'EAST NORTHAMPTONSHIRE',
 'KETTERING',
 'NORTHAMPTON',
 'SOUTH NORTHAMPTONSHIRE',
 'WELLINGBOROUGH'}

'laua_name values only in 2023:'

{'NORTH NORTHAMPTONSHIRE', 'WEST NORTHAMPTONSHIRE'}

Some values in laua_name are missing. However, in storing data in MongoDB, advantage was taken of its flexible structure, allowing documents to have different fields. Therefore, filling in missing values with null was not chosen. 

### Store the data

In [52]:
MONGO_CONNECTION_STRING = f"mongodb://localhost:27017/"
print(f"MONGO_CONNECTION_STRING = {MONGO_CONNECTION_STRING}")

MONGO_CONNECTION_STRING = mongodb://localhost:27017/


In [53]:
from pymongo import MongoClient
mongo_client = MongoClient(MONGO_CONNECTION_STRING)
DB_NAME = "Q1_TMA02_TM351"  
print(f"DB_NAME = {DB_NAME}")

mongo_db = mongo_client[DB_NAME]

DB_NAME = Q1_TMA02_TM351


In [54]:
import os
from pymongo import MongoClient


# List of DataFrames and corresponding years
dataframes = [df2019, df2020, df2021, df2022, df2023]
years = [2019, 2020, 2021, 2022, 2023]

for df, year in zip(dataframes, years):
    # Drop the 'year' column if it already exists
    if 'year' in df.columns:
        df = df.drop(columns=['year'])

    # Add a field for the year
    df['year'] = year

    # Convert DataFrame to list of dictionaries
    records = df.to_dict(orient='records')

    # Specify the collection name based on the year
    collection_name = f'data_{year}'

    # Insert data into MongoDB
    mongo_db[collection_name].insert_many(records)



/tmp/ipykernel_412/1316622539.py:18: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  records = df.to_dict(orient='records')


As you prepared the data for entry into mongoDB you would find that the dataset contained dirty and missing data. Give three examples where you identified problems with the data. Explain how you resolved these problems and what the implications might be when you analyse the data.

*(5 marks)*

**Write your answer in this markdown cell**


In the process of preparing data for entry into MongoDB, three instances of data quality issues were identified.

Firstly, within each CSV dataset representing a specific year, the 'year' information was absent. To address this, the integration process ensured the addition of 'year' information.

Secondly, there are missing values. While columns related to 'Gigabit' were present from 2020 to 2023, there was a gap in the information for the year 2019. Regarding the 'laua_name' column, some local authorities are missing in each dataframe. However, in storing data in MongoDB, advantage was taken of its flexible structure, allowing documents to have different fields. Therefore, filling in missing values with null was not chosen. This decision aligns with MongoDB's ability to handle sparse data effectively, providing flexibility for future changes in the data structure without disrupting existing documents.

Thirdly, a discrepancy was observed in the column names for 'SFBB' and 'UFBB' between 2019 and the subsequent years (2020-2023). To establish consistency, the column names in 2019 were changed to match those from 2020 to 2023.